**Exercice 3**

Include batches

In [ ]:
def _main_ (n):

  #!/usr/bin/env python3
  # -*- coding: utf-8 -*-

  import pandas as pd
  from scipy.stats import zscore
  import torch as torch
  import numpy as np
  from torch.utils.data import random_split, DataLoader, TensorDataset

  np.random.seed(42)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  #Use pandas to load the iris dataset
  url = 'https://raw.githubusercontent.com/nourchene-bargaoui/Exercice_1_ML/main/iris.csv'
  dfData = pd.read_csv(url)

  #Create dummy variables for the classes
  np_x = dfData[list(dfData.columns)[0:-1]].apply(zscore).to_numpy();
  np_y = pd.get_dummies(dfData['variety']).to_numpy(); #Converts categorical variable into dummy variables

  # 3 classes = setosa, Versicolor, Virginica
  n_classes = 3

  n_features = np_x.shape[1]
  x_train = np_x
  y_train = np_y

  #Define pytorch tensors for the dataset using torch.tensor
  t_x_train = torch.tensor(x_train,requires_grad=False,device=device);
  t_y_train = torch.tensor(y_train,requires_grad=False,device=device);
  data = TensorDataset(t_x_train,t_y_train)
  print('t_x_train:', t_x_train)
  print('t_y_train:',t_y_train)

  # Split to Train and Test sets using random_split
  rows_number = len(t_x_train)    # The size of our dataset or the number of rows in excel table.
  train_data_split = int(rows_number*0.8)  #80% train
  test_data_split = int(rows_number*0.2)   #20% test
  train_set, test_set = random_split(data, [train_data_split, test_data_split])

  # Create Dataloader to read the data within batch sizes.
  train_batch_size = 20
  train_loader = DataLoader(dataset = train_set, batch_size = train_batch_size, shuffle = True)
  test_loader = DataLoader(dataset = test_set, batch_size = len(test_set), shuffle = True)

  #define starting value of weights W for gradient descent
  init_std_dev = 0.01;
  initialW=init_std_dev*np.random.randn(n_features,n_classes)

  # create a PyTorch tensor variable for W and b.
  # we will be optimizing over W, b, finding their best value using gradient descent, so we need gradient enabled
  W = torch.tensor(initialW,requires_grad=True,device=device);
  b = torch.zeros((1,n_classes),requires_grad=True,device=device);

  learning_rate = 0.01;
  #Choosing between 2 optimizers
  if (n==1):
    optimizer = torch.optim.SGD([W,b],lr=learning_rate) # this optimizer will do gradient descent for us

  if (n==2):
    optimizer = torch.optim.Adam([W,b],lr=learning_rate)

  n_epochs = 10000

  run_loss = 0
  loss = []
  training_accuracy = []

  total_batches = len(train_loader.dataset)
  for i in range(n_epochs):
      accuracy = 0
      batches = 0
      for batch_id,(X,Y) in enumerate(train_loader):
          batches += 1
          # clear previous gradient calculations
          optimizer.zero_grad();

          # calculate model predictions
          linear_predictions = torch.matmul(X,W)+b

          # Pass linear_predictions to softmax
          softmax_lin_pred = torch.div(torch.exp(linear_predictions), torch.sum(torch.exp(linear_predictions), axis = 1, keepdim = True))

          #calculate Cross entropy loss
          risk = -torch.mean(torch.mul(Y,torch.log(softmax_lin_pred)))

          #propagate them back
          risk.backward();

          # use the gradient to change W, b
          optimizer.step();
          run_loss +=risk.item()
          # calculate accuracy (on the training set!)
          true_class = np.argmax(Y.detach().cpu().numpy(),axis=1)
          pred_class = np.argmax(softmax_lin_pred.detach().cpu().numpy(),axis=1)
          accuracy += np.count_nonzero(true_class == pred_class)/pred_class.shape[0];
          error = 1.0 - accuracy;

      loss.append(run_loss/batches)
      training_accuracy.append(accuracy/batches)
      print('Epoch: ', i,'loss: ', risk.item(), 'accuracy',accuracy/batches)
      print('')
  # model Testing

  # calculate model predictions

  for batch_id, (test_x, test_y) in enumerate(test_loader):
      linear_predictions = torch.matmul(test_x,W)+b
      softmax_lin_pred = torch.div(torch.exp(linear_predictions), torch.sum(torch.exp(linear_predictions), axis = 1, keepdim = True))

      #calculate loss
      risk_loss = -torch.mean(torch.mul(test_y,torch.log(softmax_lin_pred)))

      #calculate accuracy (on the test set!)

      true_class = np.argmax(test_y.detach().cpu().numpy(),axis=1)
      pred_class = np.argmax(softmax_lin_pred.detach().cpu().numpy(),axis=1)
      accuracy = np.count_nonzero(true_class == pred_class)/pred_class.shape[0];
      error = 1.0 - accuracy;
      print('Test Accuracy:')
      print('Epoch: ', i,'loss: ', risk_loss.item(), 'accuracy', accuracy)


In [ ]:
#Learning rate = 0.001 , optimizer = SGD , epochs=10000, standard deviation = 0.01
_main_(1)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Epoch:  7501 loss:  0.033706455819490104 accuracy 0.9750000000000001

Epoch:  7502 loss:  0.039809869794400245 accuracy 0.9750000000000001

Epoch:  7503 loss:  0.03920768216603543 accuracy 0.975

Epoch:  7504 loss:  0.044525099816830095 accuracy 0.9750000000000001

Epoch:  7505 loss:  0.01931092089340424 accuracy 0.9750000000000001

Epoch:  7506 loss:  0.041939535833558514 accuracy 0.975

Epoch:  7507 loss:  0.04273476216735023 accuracy 0.975

Epoch:  7508 loss:  0.027106172704709077 accuracy 0.9750000000000001

Epoch:  7509 loss:  0.03225293376966196 accuracy 0.975

Epoch:  7510 loss:  0.01874244439539013 accuracy 0.975

Epoch:  7511 loss:  0.03806072175944565 accuracy 0.9750000000000001

Epoch:  7512 loss:  0.04411433228848998 accuracy 0.9750000000000001

Epoch:  7513 loss:  0.06422678271873271 accuracy 0.9750000000000001

Epoch:  7514 loss:  0.0382274892101178 accuracy 0.9750000000000001

Epoch:  7515 loss

In [ ]:
#Learning rate = 0.001 , optimizer = Adam , epochs=10000, standard deviation = 0.01
_main_(2)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Epoch:  7501 loss:  0.00022141589878313744 accuracy 1.0

Epoch:  7502 loss:  1.311960846470983e-06 accuracy 1.0

Epoch:  7503 loss:  0.01174232989725732 accuracy 1.0

Epoch:  7504 loss:  0.00044353582463738307 accuracy 1.0

Epoch:  7505 loss:  2.557876867370997e-05 accuracy 1.0

Epoch:  7506 loss:  0.003404318700704697 accuracy 1.0

Epoch:  7507 loss:  0.0002677812975478577 accuracy 1.0

Epoch:  7508 loss:  0.000437770907005478 accuracy 1.0

Epoch:  7509 loss:  0.011785103991572265 accuracy 1.0

Epoch:  7510 loss:  0.010999058112325976 accuracy 1.0

Epoch:  7511 loss:  0.007088730861452131 accuracy 1.0

Epoch:  7512 loss:  0.011895921370685951 accuracy 1.0

Epoch:  7513 loss:  8.918662166668451e-06 accuracy 1.0

Epoch:  7514 loss:  0.014036306777124384 accuracy 1.0

Epoch:  7515 loss:  0.008878176476521468 accuracy 1.0

Epoch:  7516 loss:  0.01353368981513395 accuracy 1.0

Epoch:  7517 loss:  0.00588638404500

# **Report**

In this exercice, we did a split of the Iris dataset into a training set and a test set.
We observed the behavior of the two Optimizers and we can tell that loss is way more smaller with Adam than with SGD.
We note that the batch size and learning rate really affect the performance of the Optimizer and we can also note that Adam is faster and gives a smaller loss than SGD.